In [ ]:
#!pip install --user cmsdials[pandas,tqdm]

In [ ]:
import os, sys, time
import warnings
warnings.filterwarnings("ignore", module="uhi")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use("ROOT")
from cmsdials.auth.bearer import Credentials
from cmsdials import Dials
from cmsdials.filters import RunFilters, LumisectionFilters, MEFilters
from cmsdials.filters import LumisectionHistogram2DFilters
from utilities import *

In [ ]:
# Autentication
creds = Credentials.from_creds_file()
# workspace definition
dials = Dials(creds, workspace="csc")

In [ ]:
# Run List: 380292, 380310, 380355, 380377, 380385, 380399, 380444

In [ ]:
run_list_df = dials.run.list_all(
    RunFilters(
        run_number__gte=380292,
        run_number__lte=380446
    )
).to_pandas()

In [ ]:
run_list = run_list_df["run_number"].to_numpy()

In [ ]:
my_runs = [380292, 380310, 380355, 380377, 380385, 380399, 380444]
my_runs_new = [i for i in my_runs if i in run_list]

In [ ]:
print(my_runs)
print(my_runs_new)

In [ ]:
from cmsdials.filters import LumisectionHistogram2DFilters
start_time = time.perf_counter()
data = []
for i, run in enumerate(my_runs_new):
    data_temp = dials.h2d.list_all(
        LumisectionHistogram2DFilters(
            me = "CSC/CSCOfflineMonitor/recHits/hRHGlobalm2",
            dataset = "/StreamExpress/Run2024D-Express-v1/DQMIO",
            run_number = run,
        ),
        enable_progress=True,
    ).to_pandas()
    data.append(data_temp)
    del data_temp
data = pd.concat(data, axis=0)
end_time = time.perf_counter()
print(f"Execution time: {end_time - start_time:.4f} seconds")

In [ ]:
data.to_parquet("data.parquet")

In [ ]:
data = pd.read_parquet("data.parquet")

In [ ]:
# Per-LS meta-info retrieved form OMS and RunRegistry
lumi_info = pd.read_parquet("perLSinfo.parquet")

In [ ]:
monitoring_elements = pd.merge(data, lumi_info, on=['run_number', 'ls_number'], how='left') 
monitoring_elements = monitoring_elements[
    (monitoring_elements["beams_stable"] == True) &
    (monitoring_elements["cscm_ready"] == True) &
    (monitoring_elements["cms_active"] == True) &
    (monitoring_elements["beam_present"] == True) &
    (monitoring_elements["physics_flag"] == True) &
    (monitoring_elements["cscSTANDBY"] == 0) &
    (monitoring_elements["cscBAD"] == 0) &
    (monitoring_elements["cscGOOD"] != 0) &
    (monitoring_elements["mean_lumi"] > 2) &
    (monitoring_elements["class"].str.contains("Collisions", na=False))
]
monitoring_elements = monitoring_elements.sort_values(by=['run_number', 'ls_number']).reset_index()
monitoring_elements = monitoring_elements.drop(columns=["index"])

In [ ]:
monitoring_elements["group"] = groupbylumi(monitoring_elements, 300)

In [ ]:
summed_data = monitoring_elements.groupby("group").apply(sum_imgs)
summed_data = summed_data[summed_data["lumi"]>300]
summed_data = summed_data.reset_index()

In [ ]:
np.unique(summed_data["run_min"])

In [ ]:
Show2Dimg(summed_data["img"][0]);

In [ ]:
# Apply the function to each row of the DataFrame
summed_data["new_img"] = summed_data.apply(
    lambda row: mask_outside_radius(row["img"], center=(49.5, 49.5), max_distance=21), axis=1
)

In [ ]:
Show2Dimg(summed_data["new_img"][0]);

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms


In [ ]:
model = torch.jit.load("model.pth")

In [ ]:
start_time = time.perf_counter()
print(f"Number of immages to be processed: {len(summed_data)}")
tensor = torch.stack([torch.tensor(m, dtype=torch.float32).unsqueeze(0) for m in summed_data["new_img"]])
loader = DataLoader(dataset=tensor, batch_size=128, num_workers=8, shuffle=False)

imgs, reco_imgs, loss_imgs = [], [], []

model.eval()
with torch.no_grad():
    for img_batch in loader:
        reco_img_batch = model(img_batch)
        img_loss_batch = (img_batch - reco_img_batch)[:, 0] / reco_img_batch[:, 0]
        #img_loss_batch = F.l1_loss(img_batch, reco_img_batch, reduction='none')[:, 0] / reco_img_batch[:, 0]
        loss_imgs.extend(img_loss_batch.numpy())
        reco_imgs.extend(reco_img_batch[:, 0].numpy())
        imgs.extend(img_batch[:, 0].numpy())
        print(".", end="", flush=True)
end_time = time.perf_counter()
print(f"\nExecution time: {end_time - start_time:.4f} seconds")

In [ ]:
Show2Dimg(imgs[114])

In [ ]:
binary_matrix = (np.mean(np.sum([imgs, reco_imgs], axis=0), axis=0) != 0)
rebin_loss_imgs = [rebin_image(image, binary_matrix) for image in loss_imgs]

In [ ]:
Show2DLoss(rebin_loss_imgs[0])

In [ ]:
summed_data["Max"] = [np.nanmax(matrix) for matrix in rebin_loss_imgs]
summed_data["Min"] = [np.nanmin(matrix) for matrix in rebin_loss_imgs]

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(summed_data["Max"], bins=50, range=(0,4), alpha=0.7, label="Max Values", color='red')
plt.xlabel('Max Loss')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(summed_data["Min"], bins=50, range=(-1.3,0.2),alpha=0.7, label="Min Values", color='blue')
plt.xlabel('Min Loss')
plt.legend()
plt.show()

In [ ]:
summed_data_filter = summed_data[(summed_data["Max"]>0.86) | (summed_data["Min"]<-0.52)]

In [ ]:
np.unique(summed_data_filter["run_max"])

In [ ]:
summed_data_filter 

In [ ]:
#plot_LSs(monitoring_elements, 380470, (1419, 1447))